In [ ]:
# !pip install import_ipynb --quiet
# !git clone https://github.com/gmshroff/aml-public.git
# %cd aml-public

In [ ]:
import import_ipynb
import utils
import models
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pickle
import warnings
from ucimlrepo import fetch_ucirepo
warnings.filterwarnings('ignore') 
from matplotlib import pyplot as plt

In [ ]:
import numpy as np
import torch
from torch import nn
from torch import optim

In [ ]:
import import_ipynb
import models

In [ ]:
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
Xf = iris.data.features 
yf = iris.data.targets 
classes=yf['class'].unique().tolist()

In [ ]:
perm = np.random.permutation(150).tolist()

In [ ]:
Xf,yf=Xf.iloc[perm],yf.iloc[perm]

In [ ]:
def encode_target(t,classes=classes):
    return classes.index(t)

In [ ]:
yf['label']=yf['class'].apply(encode_target)

In [ ]:
train_n=100

In [ ]:
X=torch.tensor(Xf.iloc[0:train_n].values, dtype=torch.float32)

In [ ]:
y=torch.tensor(yf.iloc[0:train_n]['label'].values,dtype=torch.float32)

In [ ]:
Xtest=torch.tensor(Xf.iloc[train_n:].values,dtype=torch.float32)
ytest=torch.tensor(yf[train_n:]['label'].values,dtype=torch.float32)

In [ ]:
def one_hot(y,n=3):
    oh=torch.zeros(y.shape[0],n)
    oh[torch.arange(y.shape[0]), y.long()]=1
    return oh

In [ ]:
y_train=one_hot(y)

In [ ]:
y_train

In [ ]:
class DotSoft(nn.Module):
    def __init__(self,x_train,y_train):
        super(DotSoft,self).__init__()
        self.x_train=x_train
        self.y_train=y_train
        self.S=nn.Softmax(dim=-1)
    def __call__(self,X):
        return self.S(X@self.x_train.transpose(1,0)/(X.norm()*self.x_train.norm()))@self.y_train


In [ ]:
dS=DotSoft(X,y_train)

In [ ]:
Xtest.shape

In [ ]:
ytest

In [ ]:
dS(Xtest)

In [ ]:
models.accuracy(dS,Xtest,ytest)

In [ ]:
class SoftNN(nn.Module):
    def __init__(self,x_train,y_train,d=10,lr=1e-3,weight_decay=0):
        super(SoftNN,self).__init__()
        self.x_train=x_train
        self.y_train=y_train
        self.W=torch.rand(x_train.shape[1],d,requires_grad=True)
        self.S=nn.Softmax(dim=-1)
        self.optimizer = optim.Adam([self.W],lr=lr,weight_decay=weight_decay)
    def __call__(self,X):
        Z=self.x_train@self.W
        V=X@self.W
        return self.S(V@Z.transpose(1,0)/(V.norm()*Z.norm()))@self.y_train


In [ ]:
snn=SoftNN(X,y_train)

In [ ]:
snn,losses,accs=models.TrainVals(snn,X.numpy(),y,verbose=True)

In [ ]:
models.accuracy(snn,Xtest,ytest)